In [3]:
import pandas as pd

In [ ]:
txns = pd.read_parquet("/Users/andrewtaylor/Google Drive/01-Banking_Ins/Budget_Data/data/raw_transactions.parquet")
accts = pd.read_parquet("/Users/andrewtaylor/Google Drive/01-Banking_Ins/Budget_Data/data/account_info.parquet")

In [ ]:
txns.head()

In [ ]:
accts.head()

In [ ]:
ptxns = pd.read_parquet("/Users/andrewtaylor/Google Drive/01-Banking_Ins/Budget_Data/data/processed_transactions.parquet")
ptxns_2 = pd.read_parquet("/Users/andrewtaylor/Google Drive/01-Banking_Ins/Budget_Data/data/processed_transactions_2.parquet")

In [ ]:
ptxns

In [ ]:
ptxns.count()

In [ ]:
ptxns_2.count()

In [ ]:
gtxns = pd.read_parquet("/Users/andrewtaylor/Google Drive/01-Banking_Ins/Budget_Data/data/gsheet_transactions.parquet")

In [ ]:
gtxns.head()

In [ ]:
gtxns.shape

In [ ]:
cat_data = pd.read_csv("/Users/andrewtaylor/Google Drive/01-Banking_Ins/Budget_Data/data/past_budget_data.csv")

In [ ]:
cat_data = pd.read_csv(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\past_budget_data.csv")
(
    cat_data
    .reset_index().rename(columns={'index': 'txn_id', 'txn_desc':'txn_name'})
    .astype({'txn_id':'str'})
    .to_parquet(r'G:\My Drive\01-Banking_Ins\Budget_Data\data\cat_data.parquet')
)

In [ ]:
pd.read_parquet('/Users/andrewtaylor/Google Drive/01-Banking_Ins/Budget_Data/data/raw_balances.parquet')

In [ ]:
import datetime
start_date = datetime.date(2023, 1, 1)
pd.DataFrame({'bal_date': pd.date_range(start_date, datetime.datetime.today())})

In [ ]:
processed_bals = pd.read_parquet('/Users/andrewtaylor/Google Drive/01-Banking_Ins/Budget_Data/data/processed_balances.parquet')

In [ ]:
processed_bals

# Windows

In [ ]:
txns = pd.read_parquet(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\raw_transactions.parquet")
# txns[~txns['account_id'].str.contains('NA_CHASE')].to_parquet(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\raw_transactions.parquet")

In [ ]:
p_txns = pd.read_parquet(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\processed_transactions.parquet")
p_txns.head()

In [ ]:
g_txns = pd.read_parquet(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\gsheet_transactions.parquet")
g_txns.head()

In [ ]:
account_info = pd.read_parquet(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\account_info.parquet")
account_info.head(20)

In [ ]:
cat_data = pd.read_csv(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\past_budget_data.csv")
(
    cat_data
    .reset_index().rename(columns={'index': 'txn_id', 'txn_desc':'txn_name'})
    .astype({'txn_id':'str'})
    .to_parquet(r'G:\My Drive\01-Banking_Ins\Budget_Data\data\cat_data.parquet')
)

# Remove Repeats from processed and raw transactions

In [4]:
txns = pd.read_parquet(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\raw_transactions.parquet")
p_txns = pd.read_parquet(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\processed_transactions.parquet")
g_txns = pd.read_parquet(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\gsheet_transactions.parquet")

In [5]:
manual_remove_txns = [
    'RMA TOLL','RMA TOLL USA','APPLE.COM/BILL','APPLE.COM/BILL CUPERTINO USA','PARAMOUNT+ WEST HOLLYWOO','Kindle Svcs*MB9YS6UJ3 WA Digital Account Number XXXXXXXXXXXX8143',
    'Kindle Svcs*R20SU0G00 WA Digital Account Number XXXXXXXXXXXX8143','Kindle Svcs USA','USA TECHNOLOGIES','USA TECHNOLOGIES','AMAZON.COM*RT8MR0A80',
    'APPLE.COM/BILL','PALMS CAR WASH','PALMS CAR WASH AUSTIN USA','PANGOBOOKS HENDERSONVILL TN Digital Account Number XXXXXXXXXXXX4514',
    'PANGOBOOKS +16152859422 USA','APPLE.COM/BILL','Nintendo CA1121788447','BOOKSAMILLION.COM','BOOKSAMILLION.COM BIRMINGHAM USA',
    'ETSY.COM - ARTISANFASH BROOKLYN USA','ETSY.COM - ARTISANFASH 8446593879 NY Digital Account Number XXXXXXXXXXXX5186','PANGOBOOKS HENDERSONVILL TN Digital Account Number XXXXXXXXXXXX4514',
    'PANGOBOOKS +16152859422 USA','Nike.com 1800806645','Nike.com 1800806645 USA'
]

In [6]:
g_txns[g_txns['txn_name']=='180 VILANO GRILL']

,txn_date,txn_amount,txn_name,account_name_parent,txn_cat,txn_cat_flag,txn_id
59,2024-01-05,31.61,180 VILANO GRILL,Chase,Restaurants,False,NA_CHASE_30
64,2024-01-01,31.61,180 VILANO GRILL,Chase,Restaurants,False,NA_CHASE_39


In [7]:
exclude_ids = ['NA_CHASE_65',
 'NA_CHASE_66',
 'NA_CHASE_67',
 'NA_CHASE_68',
 'NA_CHASE_69',
 'NA_CHASE_70',
 'NA_CHASE_71',
 'NA_CHASE_72',
 'NA_CHASE_73',
 'NA_CHASE_74']
dupe_cols = [c for c in txns.columns if c not in ['raw_data', 'txn_id', 'create_dt', 'archive_dt', 'current', 'txn_cat_plaid', 'txn_name_plaid', 'txn_cat_plaid_dtl', 'txn_date']]
txns_1 = txns[~txns['txn_id'].isin(exclude_ids)].drop_duplicates(subset='txn_id')
txns_2 = (
    txns_1
    .assign(count=lambda x: x.groupby(dupe_cols)['txn_id'].transform('count'))
)
print(txns_2['count'].value_counts())
txns_3 = txns_2[txns_2['count']==1].drop('count', axis=1)
txns_4 = txns_3[~txns_3['txn_name'].isin(manual_remove_txns)]

1    242
2     14
Name: count, dtype: int64


In [8]:
p_dupe_cols = [c for c in p_txns.columns if c not in ['txn_cat', 'txn_cat_flag', 'txn_id', 'txn_date', 'txn_date']]
p_txns_1 = p_txns[~p_txns['txn_id'].isin(exclude_ids)].sort_values('txn_cat_flag', ascending=False).drop_duplicates(subset='txn_id')
p_txns_2 = (
    p_txns_1
    .assign(count=lambda x: x.groupby(p_dupe_cols)['txn_id'].transform('count'))
)
print(p_txns_2['count'].value_counts())
p_txns_3 = p_txns_2[p_txns_2['count']==1].drop('count', axis=1)
p_txns_4 = p_txns_3[~p_txns_3['txn_name'].isin(manual_remove_txns)]

1    242
2     14
Name: count, dtype: int64


In [9]:
g_txns[g_txns['txn_name']=='MEMBERSHIP FEE']

,txn_date,txn_amount,txn_name,account_name_parent,txn_cat,txn_cat_flag,txn_id
74,2024-01-29,695,MEMBERSHIP FEE,Amex_Natalie,Payment - N/A,True,4LA3dvkPJNSkLZ5j3Kq5sY90yy6911FR8epR8


In [12]:
g_dupe_cols = [c for c in g_txns.columns if c not in ['txn_cat', 'txn_cat_flag', 'txn_id', 'txn_date']]
g_txns_1 = g_txns[~g_txns['txn_id'].isin(exclude_ids)].sort_values('txn_cat_flag', ascending=False).drop_duplicates(subset='txn_id')
g_txns_2 = (
    g_txns_1
    .assign(count=lambda x: x.groupby(g_dupe_cols)['txn_id'].transform('count'))
)
print(g_txns_2['count'].value_counts())
g_txns_3 = g_txns_2[g_txns_2['count']==1].drop('count', axis=1)
g_txns_4 = g_txns_3[~g_txns_3['txn_name'].isin(manual_remove_txns)]

1    240
2     12
3      3
Name: count, dtype: int64


In [13]:
txns_4.to_parquet(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\raw_transactions.parquet")
p_txns_4.to_parquet(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\processed_transactions.parquet")
g_txns_4.to_parquet(r"G:\My Drive\01-Banking_Ins\Budget_Data\data\gsheet_transactions.parquet")